In [6]:
# importing libraries
import os
os.environ["OMP_NUM_THREADS"] = "8"
import warnings
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak")
import pandas as pd
import numpy as np


# Loading the dataset
df = pd.read_excel(r"C:\Users\AMD\Downloads\Amazon_Customer_Purchase_Data (2).xlsx")

In [7]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Customer_ID                 2000 non-null   int64         
 1   Customer_Name               1900 non-null   object        
 2   Age                         1900 non-null   float64       
 3   Gender                      2000 non-null   object        
 4   Location                    2000 non-null   object        
 5   Product_Category            2000 non-null   object        
 6   Product_ID                  2000 non-null   int64         
 7   Purchase_Date               2000 non-null   datetime64[ns]
 8   Purchase_Amount             1800 non-null   float64       
 9   Payment_Method              2000 non-null   object        
 10  Rating                      1860 non-null   float64       
 11  Feedback_Comments           1099 non-null   object      

In [8]:
df


,Customer_ID,Customer_Name,Age,Gender,Location,Product_Category,Product_ID,Purchase_Date,Purchase_Amount,Payment_Method,Rating,Feedback_Comments,Customer_Lifetime_Value,Loyalty_Score,Discount_Applied,Return_Status,Customer_Segment,Preferred_Shopping_Channel
0,17270,John,56.0,Other,New York,Books,674,2020-01-01 00:00:00,491.643012,Cash,NaN,NaN,3673.712747,60,No,No,Regular,In-store
1,10860,Eve,33.0,Other,Houston,Home Appliances,393,2020-01-01 01:00:00,144.326722,Cash,5.0,Good,2103.060388,29,Yes,Yes,New,In-store
2,15390,John,50.0,Female,Houston,Clothing,995,2020-01-01 02:00:00,109.301892,Bank Transfer,NaN,NaN,899.115059,92,No,No,VIP,Online
3,15191,Eve,66.0,Other,San Francisco,Electronics,405,2020-01-01 03:00:00,226.655516,Bank Transfer,2.0,Excellent,2591.137716,62,Yes,Yes,Regular,Online
4,15734,Eve,38.0,Female,New York,Toys,353,2020-01-01 04:00:00,37.851880,Bank Transfer,2.0,NaN,548.620397,80,No,Yes,Regular,Both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,10661,Charlie,61.0,Male,Los Angeles,Home Appliances,989,2020-03-24 03:00:00,168.291356,Credit Card,5.0,Excellent,1058.502176,3,Yes,No,New,Online
1996,13374,Eve,41.0,Female,Chicago,Books,409,2020-03-24 04:00:00,237.252493,Bank Transfer,5.0,Average,1581.739382,79,Yes,No,Regular,Online
1997,19208,Alice,58.0,Female,Houston,Toys,617,2020-03-24 05:00:00,213.388283,PayPal,4.0,Good,1155.797595,76,Yes,No,Regular,In-store
1998,10207,John,18.0,Female,San Francisco,Books,113,2020-03-24 06:00:00,83.056883,Bank Transfer,2.0,NaN,1155.489570,32,No,No,New,Online


In [10]:
# Fixing Data Types
df['Purchase_Date'] = pd.to_datetime(df['Purchase_Date'])

# Handling the Missing Values

# Fill numeric columns with mean
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Purchase_Amount'] = df['Purchase_Amount'].fillna(df['Purchase_Amount'].mean())
df['Customer_Lifetime_Value'] = df['Customer_Lifetime_Value'].fillna(df['Customer_Lifetime_Value'].mean())
df['Rating'] = df['Rating'].fillna(df['Rating'].mean())

#Fill categorical with mode
df['Payment_Method'] = df['Payment_Method'].fillna(df['Payment_Method'].mode()[0])


In [11]:
df


,Customer_ID,Customer_Name,Age,Gender,Location,Product_Category,Product_ID,Purchase_Date,Purchase_Amount,Payment_Method,Rating,Feedback_Comments,Customer_Lifetime_Value,Loyalty_Score,Discount_Applied,Return_Status,Customer_Segment,Preferred_Shopping_Channel
0,17270,John,56.0,Other,New York,Books,674,2020-01-01 00:00:00,491.643012,Cash,2.978495,NaN,3673.712747,60,No,No,Regular,In-store
1,10860,Eve,33.0,Other,Houston,Home Appliances,393,2020-01-01 01:00:00,144.326722,Cash,5.000000,Good,2103.060388,29,Yes,Yes,New,In-store
2,15390,John,50.0,Female,Houston,Clothing,995,2020-01-01 02:00:00,109.301892,Bank Transfer,2.978495,NaN,899.115059,92,No,No,VIP,Online
3,15191,Eve,66.0,Other,San Francisco,Electronics,405,2020-01-01 03:00:00,226.655516,Bank Transfer,2.000000,Excellent,2591.137716,62,Yes,Yes,Regular,Online
4,15734,Eve,38.0,Female,New York,Toys,353,2020-01-01 04:00:00,37.851880,Bank Transfer,2.000000,NaN,548.620397,80,No,Yes,Regular,Both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,10661,Charlie,61.0,Male,Los Angeles,Home Appliances,989,2020-03-24 03:00:00,168.291356,Credit Card,5.000000,Excellent,1058.502176,3,Yes,No,New,Online
1996,13374,Eve,41.0,Female,Chicago,Books,409,2020-03-24 04:00:00,237.252493,Bank Transfer,5.000000,Average,1581.739382,79,Yes,No,Regular,Online
1997,19208,Alice,58.0,Female,Houston,Toys,617,2020-03-24 05:00:00,213.388283,PayPal,4.000000,Good,1155.797595,76,Yes,No,Regular,In-store
1998,10207,John,18.0,Female,San Francisco,Books,113,2020-03-24 06:00:00,83.056883,Bank Transfer,2.000000,NaN,1155.489570,32,No,No,New,Online


In [12]:
# Removing the Duplicates
df.drop_duplicates(subset=['Customer_ID', 'Purchase_Date'], inplace=True)


In [13]:
# Text Consistency like..capitalizing
df['Gender'] = df['Gender'].str.capitalize()
df['Payment_Method'] = df['Payment_Method'].str.title()

In [14]:
# Handling the Outliers
from scipy.stats import zscore
df = df[(zscore(df[['Purchase_Amount', 'Customer_Lifetime_Value']]) < 3).all(axis=1)]

In [15]:
# Time-based Features
df['Purchase_Month'] = df['Purchase_Date'].dt.month
df['Purchase_Weekday'] = df['Purchase_Date'].dt.day_name()

In [16]:
# total purchases by each of the cust
purchase_freq = df.groupby('Customer_ID')['Product_ID'].count().reset_index(name='Total_Purchases')
df = df.merge(purchase_freq, on='Customer_ID')

In [17]:
# Avg Purchase Amount per Customer
avg_purchase = df.groupby('Customer_ID')['Purchase_Amount'].mean().reset_index(name='Avg_Purchase')
df = df.merge(avg_purchase, on='Customer_ID')

In [18]:
# Encoding Categoricals

df['Preferred_Shopping_Channel_Original'] = df['Preferred_Shopping_Channel']
df = pd.get_dummies(df, columns=['Gender', 'Payment_Method', 'Customer_Segment', 'Preferred_Shopping_Channel'], drop_first=True)


In [19]:
df

,Customer_ID,Customer_Name,Age,Location,Product_Category,Product_ID,Purchase_Date,Purchase_Amount,Rating,Feedback_Comments,...,Preferred_Shopping_Channel_Original,Gender_Male,Gender_Other,Payment_Method_Cash,Payment_Method_Credit Card,Payment_Method_Paypal,Customer_Segment_Regular,Customer_Segment_VIP,Preferred_Shopping_Channel_In-store,Preferred_Shopping_Channel_Online
0,17270,John,56.0,New York,Books,674,2020-01-01 00:00:00,491.643012,2.978495,NaN,...,In-store,False,True,True,False,False,True,False,True,False
1,10860,Eve,33.0,Houston,Home Appliances,393,2020-01-01 01:00:00,144.326722,5.000000,Good,...,In-store,False,True,True,False,False,False,False,True,False
2,15390,John,50.0,Houston,Clothing,995,2020-01-01 02:00:00,109.301892,2.978495,NaN,...,Online,False,False,False,False,False,False,True,False,True
3,15191,Eve,66.0,San Francisco,Electronics,405,2020-01-01 03:00:00,226.655516,2.000000,Excellent,...,Online,False,True,False,False,False,True,False,False,True
4,15734,Eve,38.0,New York,Toys,353,2020-01-01 04:00:00,37.851880,2.000000,NaN,...,Both,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,10661,Charlie,61.0,Los Angeles,Home Appliances,989,2020-03-24 03:00:00,168.291356,5.000000,Excellent,...,Online,True,False,False,True,False,False,False,False,True
1996,13374,Eve,41.0,Chicago,Books,409,2020-03-24 04:00:00,237.252493,5.000000,Average,...,Online,False,False,False,False,False,True,False,False,True
1997,19208,Alice,58.0,Houston,Toys,617,2020-03-24 05:00:00,213.388283,4.000000,Good,...,In-store,False,False,False,False,True,True,False,True,False
1998,10207,John,18.0,San Francisco,Books,113,2020-03-24 06:00:00,83.056883,2.000000,NaN,...,Online,False,False,False,False,False,False,False,False,True


In [20]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

cluster_data = df[['Purchase_Amount', 'Loyalty_Score', 'Total_Purchases']]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(cluster_data)

kmeans = KMeans(n_clusters=3, random_state=42)
df['Customer_Cluster'] = kmeans.fit_predict(scaled_data)


In [21]:
df

,Customer_ID,Customer_Name,Age,Location,Product_Category,Product_ID,Purchase_Date,Purchase_Amount,Rating,Feedback_Comments,...,Gender_Male,Gender_Other,Payment_Method_Cash,Payment_Method_Credit Card,Payment_Method_Paypal,Customer_Segment_Regular,Customer_Segment_VIP,Preferred_Shopping_Channel_In-store,Preferred_Shopping_Channel_Online,Customer_Cluster
0,17270,John,56.0,New York,Books,674,2020-01-01 00:00:00,491.643012,2.978495,NaN,...,False,True,True,False,False,True,False,True,False,2
1,10860,Eve,33.0,Houston,Home Appliances,393,2020-01-01 01:00:00,144.326722,5.000000,Good,...,False,True,True,False,False,False,False,True,False,0
2,15390,John,50.0,Houston,Clothing,995,2020-01-01 02:00:00,109.301892,2.978495,NaN,...,False,False,False,False,False,False,True,False,True,0
3,15191,Eve,66.0,San Francisco,Electronics,405,2020-01-01 03:00:00,226.655516,2.000000,Excellent,...,False,True,False,False,False,True,False,False,True,0
4,15734,Eve,38.0,New York,Toys,353,2020-01-01 04:00:00,37.851880,2.000000,NaN,...,False,False,False,False,False,True,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,10661,Charlie,61.0,Los Angeles,Home Appliances,989,2020-03-24 03:00:00,168.291356,5.000000,Excellent,...,True,False,False,True,False,False,False,False,True,1
1996,13374,Eve,41.0,Chicago,Books,409,2020-03-24 04:00:00,237.252493,5.000000,Average,...,False,False,False,False,False,True,False,False,True,2
1997,19208,Alice,58.0,Houston,Toys,617,2020-03-24 05:00:00,213.388283,4.000000,Good,...,False,False,False,False,True,True,False,True,False,2
1998,10207,John,18.0,San Francisco,Books,113,2020-03-24 06:00:00,83.056883,2.000000,NaN,...,False,False,False,False,False,False,False,False,True,0


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = df[['Age', 'Purchase_Amount', 'Loyalty_Score', 'Total_Purchases', 'Avg_Purchase']]
y = df['Customer_Lifetime_Value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

print("CLV Prediction Accuracy:", model.score(X_test, y_test))


CLV Prediction Accuracy: 0.7198240994249232


In [23]:
from sklearn.linear_model import LogisticRegression

df['Churn'] = df['Return_Status'].map({'Yes': 1, 'No': 0})

X = df[['Age', 'Purchase_Amount', 'Loyalty_Score', 'Total_Purchases', 'Avg_Purchase']]
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression()
clf.fit(X_train, y_train)

print("Churn Prediction Accuracy:", clf.score(X_test, y_test))

Churn Prediction Accuracy: 0.7775


In [24]:
top_customer = df.groupby(['Customer_ID', 'Product_Category'])['Purchase_Amount'].sum().unstack().fillna(0)

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(top_customer)
similarity_df = pd.DataFrame(similarity, index=top_customer.index, columns=top_customer.index)

def recommend_products(customer_id, top_n=3):
    if customer_id not in similarity_df.index:
        return f"Customer ID {customer_id} not found."

    similar_customers = similarity_df[customer_id].sort_values(ascending=False)[1:top_n+1]
    similar_data = df[df['Customer_ID'].isin(similar_customers.index)]
    
    # Recommend based on frequency:
    return similar_data['Product_Category'].value_counts().head(3)

a = input("Enter Any ID: ")
try:
    a = int(a)
    print(recommend_products(a))
except ValueError:
    print("Invalid input. Please enter a valid customer ID.")


Enter Any ID:  10769


Product_Category
Home Appliances    3
Books              3
Name: count, dtype: int64


In [26]:
# To export to .xlsx
df.to_excel('final_amazon_customer_data.xlsx', index=False)